In [7]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

def load_df(csv_path='~/Downloads/all \2/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


df_small = load_df(nrows=1000)


FileNotFoundError: File b'/Users/Chris/Downloads/all \x02/train_v2.csv' does not exist

In [ ]:
df_small.columns

In [ ]:
#creating new df with just the column I need and dropping old to save RAM
print(df_small['trafficSource.campaign'].unique())
print(900000 - len(df_small[df_small['trafficSource.campaign'] == "(not set)"]))

From the first 100,000 lines of the CSV, we only have 5426 rows that contain a campaign

**Ideas:**
Do a number of random samples and get the average

In [4]:
ones = []

for each in df_small.columns:
    print(str(each) + ': ' + str(df_small[each].nunique()))
    if df_small[each].nunique() == 1:
        ones.append(each)

channelGrouping: 8
customDimensions: 6
date: 328
fullVisitorId: 738367
hits: 730514
socialEngagementType: 1
visitId: 874919
visitNumber: 393
visitStartTime: 875300
device.browser: 82
device.browserSize: 1
device.browserVersion: 1
device.deviceCategory: 3
device.flashVersion: 1
device.isMobile: 2
device.language: 1
device.mobileDeviceBranding: 1
device.mobileDeviceInfo: 1
device.mobileDeviceMarketingName: 1
device.mobileDeviceModel: 1
device.mobileInputSelector: 1
device.operatingSystem: 24
device.operatingSystemVersion: 1
device.screenColors: 1
device.screenResolution: 1
geoNetwork.city: 771
geoNetwork.cityId: 1
geoNetwork.continent: 6
geoNetwork.country: 222
geoNetwork.latitude: 1
geoNetwork.longitude: 1
geoNetwork.metro: 107
geoNetwork.networkDomain: 29571
geoNetwork.networkLocation: 1
geoNetwork.region: 434
geoNetwork.subContinent: 23
totals.bounces: 1
totals.hits: 251
totals.newVisits: 1
totals.pageviews: 195
totals.sessionQualityDim: 100
totals.timeOnSite: 4101
totals.totalTransac

In [5]:
for each in ones:
    print(str(each))
    print(df_small[each].unique())
    print('\n')

socialEngagementType
['Not Socially Engaged']


device.browserSize
['not available in demo dataset']


device.browserVersion
['not available in demo dataset']


device.flashVersion
['not available in demo dataset']


device.language
['not available in demo dataset']


device.mobileDeviceBranding
['not available in demo dataset']


device.mobileDeviceInfo
['not available in demo dataset']


device.mobileDeviceMarketingName
['not available in demo dataset']


device.mobileDeviceModel
['not available in demo dataset']


device.mobileInputSelector
['not available in demo dataset']


device.operatingSystemVersion
['not available in demo dataset']


device.screenColors
['not available in demo dataset']


device.screenResolution
['not available in demo dataset']


geoNetwork.cityId
['not available in demo dataset']


geoNetwork.latitude
['not available in demo dataset']


geoNetwork.longitude
['not available in demo dataset']


geoNetwork.networkLocation
['not available in demo dataset']


to

### Remove these columns from the data 

* device.browserSize
* device.browserVersion
* device.flashVersion
* device.language
* device.mobileDeviceBranding
* device.mobileDeviceInfo
* device.mobileDeviceMarketingName
* device.mobileDeviceModel
* device.mobileInputSelector
* device.operatingSystemVersion
* device.screenColors
* device.screenResolution
* geoNetwork.cityId
* geoNetwork.latitude
* geoNetwork.longitude
* geoNetwork.networkLocation


# Looking into the keywords

In [31]:
phrases = []
keywords = df_small['trafficSource.keyword'].dropna()
for each in keywords:
    if each != '(not provided)':
        phrases.append(each)

In [34]:
print(len(phrases))

43343


In [35]:
print(phrases[:100])

['water bottle', '(Remarketing/Content targeting)', '6qEhsCssdK0z36ri', '(automatic matching)', 'Google men', '1hZbAqLCbjwfgOH7', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', '(Remarketing/Content targeting)', '1X4Me6ZKNV0zg-jV', '6qEhsCssdK0z36ri', '(Remarketing/Content targeting)', '1hZbAqLCbjwfgOH7', '6qEhsCssdK0z36ri', '1hZbAqLCbjwfgOH7', '6qEhsCssdK0z36ri', 'google online merchandise', 'google water bottle', '1hZbAqLCbjwfgOH7', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', '(User vertical targeting)', '1X4Me6ZKNV0zg-jV', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', '(automatic matching)', 'google company store', '(automatic matching)', '(Remarketing/Content targeting)', '(Remarketing/Content targeting)', '6qEhsCssdK0z36ri', '(Remarketing/Content targeting)', '(automatic matching)', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', 'https://www.googlemerchandisestore.com/', '6qEhsCssdK0z36ri', '6qEhsCssdK0z36ri', '(Remarketing/Content targeting)', '6qEhsCssdK0z36ri', 

From looking at 900,000 data points, the training data only contains keywords in 43,343 keywords inputs.

In [36]:
words = " ". join(phrases)
words = words.split(' ')
print(len(words))

95912


Separating the phrases into distinct words there are 95,912 words total.

In [38]:
word_counts = {}
for each in words:
    if each not in word_counts:
        word_counts[each] = 1
    else:
        word_counts[each] += 1

In [42]:
word_counts_sorted = sorted(word_counts.items(), key=lambda kv: kv[1], reverse=True)
print(word_counts_sorted)

[('targeting)', 16664), ('(User', 13600), ('vertical', 13600), ('(automatic', 8715), ('matching)', 8715), ('6qEhsCssdK0z36ri', 5670), ('google', 3609), ('(Remarketing/Content', 3045), ('store', 2588), ('merchandise', 1902), ('1hZbAqLCbjwfgOH7', 1434), ('Google', 1258), ('Merchandise', 1076), ('youtube', 934), ('+google', 651), ('1X4Me6ZKNV0zg-jV', 581), ('shirt', 490), ('t', 441), ('+store', 412), ('+Google', 381), ('DoubleClick', 343), ('Ad', 343), ('Exchange', 343), ('shop', 302), ('merch', 275), ('online', 242), ('+Merchandise', 231), ('+merchandise', 189), ('tube', 183), ('you', 179), ('YouTube', 172), ('https://goto.google.com/a/google.com?continue=https://goto.google.com/tyctwd-shirts', 163), ('https://www.googlemerchandisestore.com/', 145), ('+stores', 142), ('shirts', 127), ('Youtube', 122), ('buy', 119), ('tshirt', 97), ('+Products', 85), ('Store', 76), ('android', 75), ('+Swag', 72), ('for', 67), ('india', 65), ('lava', 65), ('on', 60), ('bag', 59), ('71817', 59), ('+merch', 